<a href="https://colab.research.google.com/github/Abhijitgayen/SNA_As_Msc_projects/blob/main/ok_all/LSTM%2BW2V%2BsemEval_2013_all_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Libary_enter

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [ ]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

## Some_function for prepossing

In [ ]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['tweet']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    if sentiment == 'neutral':
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['tweet'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['label'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

## Data_Set_add

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tweets_2013_At=pd.read_csv('/content/drive/MyDrive/2017_data_set/main_data/GOLD/Subtask_A/twitter-2013train-A.txt', sep="\t", header=None, names=["id","label", "tweet"]);
tweets_2013_Ad=pd.read_csv('/content/drive/MyDrive/2017_data_set/main_data/GOLD/Subtask_A/twitter-2013dev-A.txt', sep="\t", header=None, names=["id","label", "tweet"]);
tweets_2013_A=pd.read_csv('/content/drive/MyDrive/2017_data_set/main_data/GOLD/Subtask_A/twitter-2013test-A.txt', sep="\t", header=None, names=["id","label", "tweet"]); 
tweetsDatas=pd.concat([tweets_2013_At,tweets_2013_Ad]);
testData=tweets_2013_A

In [ ]:
print(tweetsDatas.head())
print(tweetsDatas.label.value_counts())
print(testData.head())
print(testData.label.value_counts())

                   id     label  \
0  264183816548130816  positive   
1  263405084770172928  negative   
2  262163168678248449  negative   
3  264249301910310912  negative   
4  262682041215234048   neutral   

                                               tweet  
0  Gas by my house hit $3.39!!!! I\u2019m going t...  
1  Theo Walcott is still shit\u002c watch Rafa an...  
2  its not that I\u2019m a GSP fan\u002c i just h...  
3  Iranian general says Israel\u2019s Iron Dome c...  
4  Tehran\u002c Mon Amour: Obama Tried to Establi...  
neutral     5325
positive    4215
negative    1798
Name: label, dtype: int64
                   id     label  \
0  264238274963451904  positive   
1  218775148495515649  positive   
2  258965201766998017   neutral   
3  262926411352903682  negative   
4  171874368908050432   neutral   

                                               tweet  
0  @jjuueellzz down in the Atlantic city, ventnor...  
1  Musical awareness: Great Big Beautiful Tomorro...  
2  On 

In [ ]:
df=tweetsDatas
# max_length is used for max sequence of input
max_length = get_max_length(df)
print(max_length)

train_X, train_Y = preprocess(df)

35


## ALL_models

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# LSTM model
model_1= Sequential()
model_1.add(LSTM(64))
model_1.add(Dense(2, activation='softmax'))
model_1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## train Models

In [ ]:
print('Train...')
model.fit(train_X, train_Y,epochs=10)

Train...
Epoch 1/10
355/355 [==============================] - 11s 24ms/step - loss: 0.6536 - accuracy: 0.8016
Epoch 2/10
355/355 [==============================] - 8s 24ms/step - loss: 0.6319 - accuracy: 0.7803
Epoch 3/10
355/355 [==============================] - 8s 23ms/step - loss: 0.6231 - accuracy: 0.7735
Epoch 4/10
355/355 [==============================] - 8s 24ms/step - loss: 0.6156 - accuracy: 0.7747
Epoch 5/10
355/355 [==============================] - 8s 24ms/step - loss: 0.6120 - accuracy: 0.7835
Epoch 6/10
355/355 [==============================] - 8s 24ms/step - loss: 0.6045 - accuracy: 0.7815
Epoch 7/10
355/355 [==============================] - 9s 25ms/step - loss: 0.5995 - accuracy: 0.7867
Epoch 8/10
355/355 [==============================] - 8s 24ms/step - loss: 0.5926 - accuracy: 0.7855
Epoch 9/10
355/355 [==============================] - 8s 24ms/step - loss: 0.5877 - accuracy: 0.7931
Epoch 10/10
355/355 [==============================] - 8s 24ms/step - loss: 0.582

In [ ]:
test_df=testData
print(test_df.label.value_counts())
max_length = get_max_length(test_df)
test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

neutral     1513
positive    1475
negative     559
Name: label, dtype: int64
111/111 - 2s - loss: 0.5866 - accuracy: 0.7857 - 2s/epoch - 18ms/step
Test score: 0.5865960121154785
Test accuracy: 0.7857344150543213


In [ ]:
print('Train...')
model_1.fit(train_X, train_Y,epochs=10)

Train...
Epoch 1/10
355/355 [==============================] - 12s 27ms/step - loss: 0.6536 - accuracy: 0.8042
Epoch 2/10
355/355 [==============================] - 8s 24ms/step - loss: 0.6309 - accuracy: 0.7725
Epoch 3/10
355/355 [==============================] - 9s 25ms/step - loss: 0.6227 - accuracy: 0.7777
Epoch 4/10
355/355 [==============================] - 9s 25ms/step - loss: 0.6135 - accuracy: 0.7811
Epoch 5/10
355/355 [==============================] - 9s 24ms/step - loss: 0.6116 - accuracy: 0.7821
Epoch 6/10
355/355 [==============================] - 9s 25ms/step - loss: 0.6022 - accuracy: 0.7862
Epoch 7/10
355/355 [==============================] - 9s 25ms/step - loss: 0.5962 - accuracy: 0.7840
Epoch 8/10
355/355 [==============================] - 8s 24ms/step - loss: 0.5913 - accuracy: 0.7886
Epoch 9/10
355/355 [==============================] - 9s 24ms/step - loss: 0.5845 - accuracy: 0.7951
Epoch 10/10
355/355 [==============================] - 9s 24ms/step - loss: 0.580

In [ ]:
test_df=testData
print(test_df.label.value_counts())
max_length = get_max_length(test_df)
test_X, test_Y = preprocess(test_df)

score, acc = model_1.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

neutral     1513
positive    1475
negative     559
Name: label, dtype: int64
111/111 - 2s - loss: 0.5927 - accuracy: 0.7922 - 2s/epoch - 18ms/step
Test score: 0.5927045941352844
Test accuracy: 0.792218804359436
